<a href="https://colab.research.google.com/github/rajaramkuberan/ANN_Coursework_Coventry_Univ/blob/main/ANN_CourseWork_GoogLeNet_Rajaram_Kuberan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANN Coursework**

## Rajaram Kuberan 

 GoogLeNet Architecture is used to perform image classification on Ketogenic and Non-Ketogenic Food items. The Keras Library is used to build the GoogLeNet CNN architecture. The training/test datas are pushed into the Github repository. 
 This is the [Github](https://github.com/rajaramkuberan/ANN_Coursework_Coventry_Univ.git) link.

In [1]:
!git clone https://github.com/rajaramkuberan/ANN_Coursework_Coventry_Univ.git

Cloning into 'ANN_Coursework_Coventry_Univ'...
remote: Enumerating objects: 4697, done.
remote: Counting objects: 100% (4697/4697), done.
remote: Compressing objects: 100% (4689/4689), done.
remote: Total 30301 (delta 8), reused 4697 (delta 8), pack-reused 25604
Receiving objects: 100% (30301/30301), 6.11 GiB | 46.35 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Checking out files: 100% (31400/31400), done.


##Importing the Necessary Libraries

In [15]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from keras.layers import concatenate
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score


##**Data Preprocessing**

Now ,let's use ImageDataGenerator class to create our train and test dataset and normalize our data. 

It's important to normalize our data because data going into our CNN to improve its overall performance. We will use the rescale parameter to scale our image pixel values from [0, 255] to [0,1].

In each generator, we specify the source directory of our images, the classes, the input image size, the batch size (how many images to process at once), and class mode.




In [16]:
# initialising the image size as 224x224
IMAGE_SIZE = [224,224]


# Setting the Path to test and train data 
test_path = '/content/ANN_Coursework_Coventry_Univ/test'
train_path = '/content/ANN_Coursework_Coventry_Univ/train'

In [17]:
# transforming train data images
train_datagen = ImageDataGenerator(rescale = 1/255)

# transforming train data images
test_datagen = ImageDataGenerator(rescale = 1/255)


In [18]:
#training data 
train_generator = train_datagen.flow_from_directory(
        '/content/ANN_Coursework_Coventry_Univ/train',  # This is the source directory for training images
        classes = ['Keto_train', 'Non_Keto_train'],
        target_size=(224, 224),  # All images are resized to 224x224
        batch_size=32,
        # Use binary labels
        class_mode='binary')

# testing data
test_generator = test_datagen.flow_from_directory(
        '/content/ANN_Coursework_Coventry_Univ/test',  # This is the source directory for test images
        classes = ['Keto_test', 'Non_Keto_Test'],
        target_size=(224, 224),  # All images are resized to 224x224
        batch_size=16,
        # Use binary labels
        class_mode='binary')

Found 24861 images belonging to 2 classes.
Found 6163 images belonging to 2 classes.


## Building the Model

In GoogLeNet architecture, there ae 4 blocks of Inception layer. Firstly to avoid repeating the layers, create the inception block function so that we can use it in the model easily.

Reference: Szegedy, Christian, et al. “Going deeper with convolutions.” Proceedings of the IEEE conference on computer vision and pattern recognition. 2015.

The above 2015 Paper is used as the reference to create the blocks. The kernel size and the convolution layers are 
exactly written in Python using Keras Library as mentioned in the paper.

### Create an Inception Block

In [19]:
# create an Inception block 
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  
  # 1st block:
  block1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', 
                 activation = 'relu')(input_layer)

  # 2nd block:
  block2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', 
                 activation = 'relu')(input_layer)
  block2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', 
                 activation = 'relu')(block2)

  # 3rd block:
  block3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', 
                 activation = 'relu')(input_layer)
  block3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', 
                 activation = 'relu')(block3)

  # 4th block:
  block4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  block4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', 
                 activation = 'relu')(block4)

  output_layer = concatenate([block1, block2, block3, block4], axis = -1)

  return output_layer

### Create a GoogLenet Layer:
 Here we will define GoogLeNet Model Layers and then return the model.




In [6]:
def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (224, 224, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', 
             activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', 
             activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', 
             activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, 
                      f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, 
                      f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, 
                      f3_conv5 = 48, f4 = 64)

  # Average Pooling Layer 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', 
              activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
  X1 = Dense(5, activation = 'softmax')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, 
                      f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, 
                      f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, 
                      f3_conv5 = 64, f4 = 64)

  # Average Pooling Layer 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', 
              activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
  X2 = Dense(1000, activation = 'softmax')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, 
                      f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
  X = Dense(1000, activation = 'softmax')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

model = GoogLeNet()

###Model Summary:
The model.summary() method call prints a summary of the NN 

In [20]:
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 109, 109, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 54, 54, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 54, 54, 64)   4160        max_pooling2d[0][0]              
__________________________________________________________________________________________

### Model Compiling/Execution:

The "output shape" column shows the transformation of the dimensions of each layer as a result of the convolution and max pooling - convolution will reduce the layer size by a bit due to padding, and max pooling will halve the output size.


Next, we'll configure the specifications for model training. We will train our model with the binary_crossentropy loss. We will use the Adam optimizer. Adam is a sensible optimization algorithm because it automates learning-rate tuning for us (alternatively, we could also use RmsProp and Adagrad for similar results). We will add accuracy to metrics so that the model will monitor accuracy during training

In [17]:
#add loss function and optimisers
#model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics =['accuracy'])

In [8]:
from tensorflow.keras.optimizers import RMSprop

# model.compile(loss='binary_crossentropy',
#               optimizer=RMSprop(lr=0.001),
#               metrics=['accuracy', tf.keras.metrics.AUC()])

from tensorflow.keras import optimizers

model.compile(optimizer = 'Adam', loss='sparse_categorical_crossentropy', metrics =['accuracy'])
#model.compile(optimizer=RMSprop(lr=0.01), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

### Fitting the Model(Training):
Let's train for 15 epochs GPU connection:


In [21]:
# epochs is 15  
history = model.fit(
      train_generator,
      steps_per_epoch=32,  
      epochs=50,
      verbose=1,
      validation_data = test_generator,
      validation_steps=16)

Epoch 1/50
32/32 [==============================] - 55s 539ms/step - loss: 7.9791 - dense_4_loss: 3.8850 - dense_1_loss: 1.0161 - dense_3_loss: 3.0781 - dense_4_accuracy: 0.5384 - dense_1_accuracy: 0.5132 - dense_3_accuracy: 0.5459 - val_loss: 2.0191 - val_dense_4_loss: 0.6582 - val_dense_1_loss: 0.6754 - val_dense_3_loss: 0.6855 - val_dense_4_accuracy: 0.6445 - val_dense_1_accuracy: 0.6445 - val_dense_3_accuracy: 0.6445
Epoch 2/50
32/32 [==============================] - 16s 506ms/step - loss: 2.1414 - dense_4_loss: 0.6971 - dense_1_loss: 0.6833 - dense_3_loss: 0.7611 - dense_4_accuracy: 0.5652 - dense_1_accuracy: 0.5770 - dense_3_accuracy: 0.5694 - val_loss: 1.9650 - val_dense_4_loss: 0.6554 - val_dense_1_loss: 0.6515 - val_dense_3_loss: 0.6582 - val_dense_4_accuracy: 0.6406 - val_dense_1_accuracy: 0.6406 - val_dense_3_accuracy: 0.6406
Epoch 3/50
32/32 [==============================] - 16s 493ms/step - loss: 2.0201 - dense_4_loss: 0.6692 - dense_1_loss: 0.6717 - dense_3_loss: 0.6793

## Metrics: Accuracy

In [22]:
model.evaluate(test_generator)

386/386 [==============================] - 68s 176ms/step - loss: 1.9600 - dense_4_loss: 0.6562 - dense_1_loss: 0.6492 - dense_3_loss: 0.6547 - dense_4_accuracy: 0.6393 - dense_1_accuracy: 0.6393 - dense_3_accuracy: 0.6393


[1.9600166082382202,
 0.6562042236328125,
 0.6491546034812927,
 0.6546580791473389,
 0.6392990350723267,
 0.6392990350723267,
 0.6392990350723267]

Now, let's calculate our ROC curve and plot it.

First, let's make predictions on our validation set. When using generators to make predictions, we must first turn off shuffle (as we did when we created validation_generator) and reset the generator:

In [23]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
preds = model.predict(test_generator,
                      verbose=1)

386/386 [==============================] - 63s 162ms/step
